# Generate data from Table 1 to be used for statistical tests by R scripts in this folder

In [ ]:
import collections
import itertools
import json
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as ss

import seaborn as sns


In [ ]:
# Ensure everything is rounded to 4 significant digits
num_sign_digits = 4
pd.options.display.float_format = ('{:,.' + str(num_sign_digits) + 'f}').format

In [ ]:
# Data in the format described more precisely in RQ3/general/4a_submit_u_evaluator_non_rtl.ipynb
input_path = "/Users/nknyazev/Documents/Delft/Thesis/temporal/data/results/best_runs.json"
rtl_results_path = "/Users/nknyazev/Documents/Delft/Thesis/temporal/data/results/rtl/offline201909.json"

In [ ]:
def add_prefix(dictionary, prefix, sep="_"):
    return {prefix + sep + k:v for k,v in dictionary.items()}
# https://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def convert_types(df):
    cols = df.columns
    types = [int, float, str]
    for col in cols:
        for t in types:
            try:
                df[col] = df[col].astype(t)
                break
            except ValueError as e:
                pass
    return df

In [ ]:
# Rename metrics + model_id, applied to df columns
def metric_renamer(col_name):
    metric_to_report_name = {
        "mrr": "MRR@20",
        "u_mrr": "UserMRR@20",
        "recall": "Recall@20",
        "u_recall": "UserRecall@20",
        "model_id": "model"
    }
    return metric_to_report_name[col_name.replace("test", "")\
        .replace("validation", "")\
        .replace("_all_", "")]

# Group data per col (e.g. per dataset) and find the highest metric among the models
def bold_col_max_per_col(df, groupby_col="Dataset"):
    df_copy = df.copy()
    # Iterate over each metric
    for c in df.columns:
        # Find idx where the group's value is the highest
        max_idx = df_copy.groupby(groupby_col)[c].transform(max) == df[c]
        # Replace those values with bold text
        df_copy[c][max_idx] = df_copy[c][max_idx].apply(lambda x: "\\textbf{" + ("{0:." + str(num_sign_digits) + "f}").format(x) + "}")
    return df_copy

# Rename datasets from working names to report names (or any other val in the index col)
def map_level(df, d, level=0, inplace=True):
    index = df.index
    index.set_levels([[d.get(item, item) for item in names] if i==level else names
                      for i, names in enumerate(index.levels)], inplace=inplace)

# Get count of cols in index + df.columns and create a string like c|c|c|c|c for alignment inside columns
def get_table_column_format(df, borders=True, positioning="c"):
    num_cols = len(df.index.names) + len(df.columns)
    table_column_format = (positioning + "|" if borders else positioning)*(num_cols-1) + positioning
    return table_column_format

In [ ]:
with open(input_path) as input_file:
    results = json.load(input_file)

In [ ]:
for k, v in results.items():
    results[k] = {**v["run_params"], **flatten(v["results"])}

In [ ]:
df = pd.DataFrame(list(results.values()))

In [ ]:
df = convert_types(df)

In [ ]:
rtl_df = pd.read_json(open(rtl_results_path)).transpose()
rtl_df.index = rtl_df.index.map(lambda x: x.split(" ")[-1])
rtl_df = rtl_df.reset_index().rename({"index": "model_id"}, axis=1)
rtl_df = rtl_df.rename(lambda x: "test_all_" + x if x != "model_id" else x, axis=1)
rtl_df[rtl_df.columns] = rtl_df[rtl_df.columns].astype(str) 
rtl_df["dataset"] = "rtl"
rtl_df = convert_types(rtl_df)

In [ ]:
# Combine results from rtl and offline data as those kept in different locations
df = pd.concat([df, rtl_df], sort=False)

In [ ]:
# Columns to keep
# cols_to_keep = [c for c in df.columns if c.startswith("test_all_")] + ["model_id"]
cols_to_keep = ["test_all_recall", "test_all_mrr", "test_all_u_recall", "test_all_u_mrr", "model_id"]
# Make MultiIndexed DF (looks nicer) and rename columns
df2 = df.copy()\
        .set_index(["dataset", "model_id"])\
        .sort_index()[[c for c in cols_to_keep if c != "model_id"]]\
        .rename(metric_renamer, axis=1)
# Rename index
df2.index.set_names(["Dataset", "model"], inplace=True)

In [ ]:
metrics = df2.columns
datasets = df2.index.get_level_values(0).unique()
for metric in metrics:
    output_path = os.path.join(output_root, "{}.csv".format(metric))
    d = df2[metric].round(5)
    to_export = d.unstack()
    to_export.rename(lambda x: "m{}".format(x), inplace=True, axis=1)
    to_export.to_csv(output_path)

In [ ]:
# Export model_ids and which equations modified
model_id_path = os.path.join(output_root, "model_ids.csv")
equation_path = os.path.join(output_root, "equations.csv")

model_id_df = pd.DataFrame(["m{}".format(x) for x in range(8)], columns=["model_id"])
equation_df = pd.DataFrame([[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1]], columns=["eq1", "eq2", "eq3"], dtype=bool)

model_id_df.to_csv(model_id_path, index=False)
equation_df.to_csv(equation_path, index=False)

In [ ]:
# Also export the whole array in non-wide format to be used by normal anova for example
non_wide_path = os.path.join(output_root, "nonwide.csv")

df3 = df2.copy()
df3.rename(lambda x: x[:-3], inplace=True, axis=1)
repeated_equations = pd.concat([equation_df]*len(df3.index.get_level_values(0).unique()))
df3 = df3.reset_index().join(repeated_equations.reset_index()).drop("index", axis=1)
df3["model"] = df3["model"].apply(lambda x: "m"+str(x))

df3.to_csv(non_wide_path, index=False)